## Why you need a good init

To understand why initialization is important in a neural net, we'll focus on the basic operation you have there: matrix multiplications. So let's just take a vector x, and a matrix a initialized randomly, then multiply them 100 times (as if we had 100 layers).



In [1]:
import torch

In [2]:
x = torch.randn(512)
a = torch.randn(512,512)

In [3]:
for i in range(100): x = a @ x

In [4]:
def stats(x: torch.Tensor)->torch.Tensor:
    print(f"mean:  {x.mean()}, std: {x.std()}")

In [5]:
stats(x)

mean:  nan, std: nan


## Activation Explosion


The problem you'll get with that is `activation explosion`: very soon, your activations will go to `nan`. We can even ask the loop to break when that first happens:

In [10]:
x = torch.randn(512)
a = torch.randn(512,512)

for i in range(100): 
    x = a @ x
    #if x.std() != x.std(): break
    if torch.isnan(x.std()): break
i

28

**How to check if a torch.Tensor is nan?**

You can always leverage the fact that `nan != nan`

```py
>>> x = torch.tensor([1, 2, np.nan])
tensor([  1.,   2., nan.])
>>> x != x
tensor([ 0,  0,  1], dtype=torch.uint8)
```

or

```py
>>> torch.isnan(x)
tensor([ 0,  0,  1], dtype=torch.uint8)
```

- [Ref:StackOverflow](https://stackoverflow.com/questions/48158017/pytorch-operation-to-detect-nans)

In [11]:
stats(x)

mean:  nan, std: nan


It only takes around 30 multiplications! On the other hand, if you initialize your activations with a scale that is too low, then you'll get another problem.

## Vanishing Activation

In [12]:
x = torch.randn(512)
a = torch.randn(512,512) * 0.01

for i in range(100): x = a @ x

stats(x)

mean:  0.0, std: 0.0


Here, every activation vanished to 0. So to avoid that problem, people have come with several strategies to initialize their weight matrices.


## Initialization Strategies 

- Use a standard deviation that will make sure `x` and `Ax` have exactly the same scale
- Use an `orthogonal matrix` to initialize the weight (orthogonal matrices have the special property that they preserve the `L2` norm, so `x` and `Ax` would have the same sum of squares in that case)
- Use `spectral normalization` on the matrix `A` (the spectral norm of A is the least possible number M such that `torch.norm(A@x) <= M*torch.norm(x)` so dividing A by this M insures you don't overflow. You can still vanish with this)


- [Ref:Paper-Spectral Normalization](https://arxiv.org/pdf/1802.05957.pdf)


## The magic number for scaling

Here we will focus on the first one, which is the `Xavier initialization`. It tells us that we should use a scale equal to `1/math.sqrt(n_in)` [$\frac{1}{\sqrt{n_{IN}}}$] where `n_in` is the number of inputs of our matrix. [`n_in = inp_dim` i.e `input dimension`]

In [13]:
import math

In [18]:
x = torch.randn(512)
a = torch.randn(512,512) / math.sqrt(512)

for i in range(100): x = a @ x
    
stats(x)

mean:  0.027958475053310394, std: 1.450333833694458


In [19]:
1/math.sqrt(512)

0.044194173824159216

And indeed it works. Note that this magic number isn't very far from the 0.01 we had earlier.

But where does it come from? It's not that mysterious if you remember the definition of the matrix multiplication. When we do `y = a @ x`, the coefficients of y are defined by

$$y_{i} = a_{i,0} x_{0} + a_{i,1} x_{1} + \cdots + a_{i,n-1} x_{n-1} = \sum_{k=0}^{n-1} a_{i,k} x_{k}
$$


or in code:

```py
y[i] = sum([c*d for c,d in zip(a[i], x)])
```

Now at the very beginning, our x vector has a mean of roughly 0. and a standard deviation of roughly 1. (since we picked it that way).

In [21]:
x = torch.randn(512)
stats(x)

mean:  -0.05562354251742363, std: 1.0106923580169678


**Note:** This is why it's extremely important to normalize your inputs in Deep Learning, the initialization rules have been designed with inputs that have a mean 0. and a standard deviation of 1.

If you need a refresher from your statistics course, the mean is the sum of all the elements divided by the number of elements (a basic average). The standard deviation shows whether the data points stay close to the mean or are far away from it. It's computed by the following formula:

$$\sigma = \sqrt{\frac{1}{n}\left[(x_{0}-\mu)^{2} + (x_{1}-\mu)^{2} + \cdots + (x_{n-1}-\mu)^{2}\right]}
$$



If we go back to `y = a @ x` and assume that we chose weights for a that also have a mean of 0, we can compute the variance of y quite easily. Since it's random, and we may fall on bad numbers, we repeat the operation 100 times.

In [22]:
mean,sqr = 0.,0.
for i in range(100):
    x = torch.randn(512)
    a = torch.randn(512, 512)
    y = a @ x
    mean += y.mean().item()
    sqr  += y.pow(2).mean().item()
mean/100,sqr/100

(-0.019350024610757827, 511.56888916015623)

Now that looks very close to the dimension of our matrix 512. And that's no coincidence! When you compute `y`, you sum 512 product of one element of `a` by one element of `x`. So what's the mean and the standard deviation of such a product of one element of `a` by one element of `x`? We can show mathematically that as long as the elements in `a` and the elements in `x` are independent, the mean is 0 and the std is 1.


This can also be seen experimentally:

In [23]:
mean,sqr = 0.,0.
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1)
    y = a*x
    mean += y.item()
    sqr  += y.pow(2).item()
mean/10000,math.sqrt(sqr/10000)

(-0.0034053195815345588, 1.0258367947839448)

Then we sum 512 of those things that have a mean of zero, and a variance of 1, so we get something that has a mean of 0, and variance of 512. To go to the standard deviation, we have to add a square root, hence math.sqrt(512) being our magic number.

If we scale the weights of the matrix a and divide them by this math.sqrt(512), it will give us a y of scale 1, and repeating the product as many times as we want and it won't overflow or vanish.

## Adding ReLU in the mix


We can reproduce the previous experiment with a ReLU, to see that this time, the mean shifts and the variance becomes `0.5`. This time the magic number will be `math.sqrt(2/512)` to properly scale the weights of the matrix.

In [28]:
mean,sqr = 0.,0.

def relu(x):
    if x<0: return 0
    else: return x.item()
    
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1)
    y = a*x
    
    # applying relu
    y = relu(y) 
    mean += y
    sqr  += y ** 2
mean/10000,sqr/10000

(0.3124287480063653, 0.4874404741692772)

We can double check by running the experiment on the whole matrix product. The variance becomes 512/2 this time:



In [29]:
mean,sqr = 0.,0.
for i in range(100):
    x = torch.randn(512)
    a = torch.randn(512, 512)
    y = a @ x
    y = y.clamp(min=0)
    mean += y.mean().item()
    sqr  += y.pow(2).mean().item()
mean/100,sqr/100

(9.145401511192322, 261.2793881225586)

Or that scaling the coefficient with the magic number gives us a scale of 1.



In [31]:
mean,sqr = 0.,0.
for i in range(100):
    x = torch.randn(512)
    a = torch.randn(512, 512) * math.sqrt(2/512)
    y = a @ x
    y = y.clamp(min=0)
    mean += y.mean().item()
    sqr  += y.pow(2).mean().item()
mean/100,math.sqrt(sqr/100)

(0.5624650683999062, 0.9974158514950515)

## What exactly is Xavier initialization?

Assigning the network weights before we start training seems to be a random process, right? We don’t know anything about the data, so we are not sure how to assign the weights that would work in that particular case. One good way is to assign the weights from a Gaussian distribution. Obviously this distribution would have zero mean and some finite variance. Let’s consider a linear neuron:

$$y = w_1 x_1 + w_2 x_2 + \dots + w_N x_N + b$$

With each passing layer, we want the variance to remain the same. This helps us keep the signal from exploding to a high value or vanishing to zero. In other words, we need to initialize the weights in such a way that the variance remains the same for x and y.


## How to perform Xavier initialization?

Just to reiterate, we want the variance to remain the same as we pass through each layer. Let’s go ahead and compute the variance of y:

$$var(y) = var(w_1 x_1 + w_2 x_2 + \dots + w_N x_N + b)$$

Let’s compute the variance of the terms inside the parentheses on the right hand side of the above equation. If you consider a general term, we have:

$$var(w_i x_i) = E(x_i)^2var(w_i) + E(w_i)^2var(x_i) + var(w_i)var(x_i)$$

Here, $E()$ stands for `expectation` of a given variable, which basically represents the mean value. We have assumed that the _inputs and weights are coming_ from a `Gaussian distribution` of zero mean. Hence the `E()` term vanishes and we get:

$$var(w_i x_i) = var(w_i)var(x_i)$$

Note that `b` is a constant and has zero variance, so it will vanish. Let’s substitute in the original equation:

$$var(y) = var(w_1)var(x_1) + \dots + var(w_N)var(x_N)$$

Since they are all identically distributed, we can write:

$$var(y) = N * var(w_i) * var(x_i)$$

So if we want the variance of y to be the same as x, then the term $N * var(w_i)$ should be equal to `1`. Hence:

$$N * var(w_i) = 1$$
$$var(w_i) = \frac{1}{N}$$

There we go! We arrived at the **Xavier initialization** formula. We need to pick the weights from a Gaussian distribution with `zero mean` and a variance of $\frac{1}{N}$, where N specifies the `inp_dim` i.e number of input neurons. This is how it’s implemented in the Caffe library. In the original paper, the authors take the average of the number input neurons and the output neurons. So the formula becomes:

$$var(w_i) = \frac{1}{N_{avg}}$$ 

where 

$$N_{avg} = \frac{N_{in} + N_{out}}2{}$$

The reason they do this is to preserve the backpropagated signal as well. But it is more computationally complex to implement. Hence we only take the number of input neurons during practical implementation.


- [Blog: Xavier Init](https://prateekvjoshi.com/2016/03/29/understanding-xavier-initialization-in-deep-neural-networks/)

- [Paper: Xavier Init](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)
- [Paper: Kaiming Init](https://arxiv.org/abs/1502.01852)